# Comprehensive API Usage Guide for Vector Database System

This notebook provides a step-by-step guide for exploring and testing all features of the Vector Database API. Use this as an interactive reference for understanding the system's capabilities.

## Table of Contents
1. [Setup and Configuration](#setup)
2. [System Status and Health](#system)
3. [Authentication](#auth)
4. [Team Management](#teams)
5. [User Management](#users)
6. [API Key Management](#apikeys)
7. [Image Management](#images)
8. [Vector Embeddings](#embeddings)
9. [Semantic Search](#search)
10. [Vector Visualization](#visualization)

<a id="setup"></a>
## 1. Setup and Configuration

First, we'll create initial admin user directly in the database, then configure our environment to interact with the API.

In [ ]:
import requests
import json

"""Always use the Admin API key: 22222222-2222-2222-2222-222222222222
The Admin team, user, and API key are created in the entrypoint:
docker-compose.yml
"""

# Configuração
BASE_URL = "http://localhost:8080"
API_KEY = "22222222-2222-2222-2222-222222222222"
HEADERS = {"Authorization": API_KEY}  # Sem o prefixo 'Bearer'

def make_request(method, endpoint):
    url = f"{BASE_URL}{endpoint}"
    print(f"Fazendo requisição: {method} {url}")
    print(f"Cabeçalhos: {HEADERS}")
    
    response = requests.request(method=method, url=url, headers=HEADERS)
    print(f"Status: {response.status_code}")
    print(f"Headers de resposta: {dict(response.headers)}")
    print(f"Resposta raw: {response.text}")
    
    try:
        return response.json()
    except Exception as e:
        print(f"Erro ao processar JSON: {e}")
        return {"text": response.text}

# Testar endpoints
print("\nTeste 1: System check")
system = make_request("GET", "/system-check")
print(json.dumps(system, indent=2))

print("\nTeste 2: Database status")
db = make_request("GET", "/debug/database")
print(json.dumps(db, indent=2))

print("\nTeste 3: Current User")
user = make_request("GET", "/api/v1/me")
print(json.dumps(user, indent=2))


Teste 1: System check
Fazendo requisição: GET http://localhost:8080/system-check
Cabeçalhos: {'Authorization': '22222222-2222-2222-2222-222222222222'}
Status: 200
Headers de resposta: {'date': 'Fri, 30 May 2025 20:43:42 GMT', 'server': 'uvicorn', 'content-length': '66', 'content-type': 'application/json'}
Resposta raw: {"status":"ok","service":"Image Management API","version":"1.0.0"}
{
  "status": "ok",
  "service": "Image Management API",
  "version": "1.0.0"
}

Teste 2: Database status
Fazendo requisição: GET http://localhost:8080/debug/database
Cabeçalhos: {'Authorization': '22222222-2222-2222-2222-222222222222'}
Status: 200
Headers de resposta: {'date': 'Fri, 30 May 2025 20:43:42 GMT', 'server': 'uvicorn', 'content-length': '4331', 'content-type': 'application/json'}
Resposta raw: {"teams_count":6,"teams":[{"id":"11111111-1111-1111-1111-111111111111","name":"Admin Team"},{"id":"fb4ef32f-8180-4862-9d89-eae0b6cfc7fe","name":"AI Research Team"},{"id":"be1d7c66-41b7-4e42-b301-d940ab4

In [ ]:
# Import necessary libraries
import requests
import json
import uuid
import os
import time
import io
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, HTML, Image as IPythonImage

# Configuration
BASE_URL = "http://localhost:8080"
API_KEY = "22222222-2222-2222-2222-222222222222"  # API key for admin access
HEADERS = {"Authorization": API_KEY}

# Helper function for making API requests
def make_request(method, endpoint, headers=None, params=None, data=None, files=None, json_data=None):
    """Make an API request and return the JSON response"""
    url = f"{BASE_URL}{endpoint}"
    if headers is None:
        headers = HEADERS.copy()
    
    print(f"Request: {method} {url}")
    
    response = requests.request(
        method=method,
        url=url,
        headers=headers,
        params=params,
        data=data,
        files=files,
        json=json_data,
        timeout=10  # 10 second timeout
    )
    
    print(f"Status: {response.status_code}")
    
    # Try to parse as JSON
    try:
        json_response = response.json()
        return json_response
    except:
        return response.text

# Helper function to download images from URLs
def download_image(url, filename):
    """Download an image from a URL and save it locally"""
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            with open(filename, 'wb') as f:
                f.write(response.content)
            return True
        else:
            print(f"Failed to download image, status code: {response.status_code}")
            return False
    except Exception as e:
        print(f"Error downloading image: {str(e)}")
        return False

print("Setup complete. Environment configured for API testing.")

  Using cached contourpy-1.3.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.58.1-cp310-cp310-macosx_10_9_universal2.whl.metadata (106 kB)
  Using cached kiwisolver-1.4.8-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.2 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 12.2 MB/s eta 0:00:00a 0:00:01
Using cached contourpy-1.3.2-cp310-cp310-macosx_11_0_arm64.whl (253 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.58.1-cp310-cp310-macosx_10_9_universal2.whl (2.7 MB)
Using cached kiwisolver-1.4.8-cp310-cp310-macosx_11_0_arm64.whl (65 kB)
Using cached pyparsing-3.2.3-py3-none-any.whl (111 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Setup

<a id="system"></a>
## 2. System Status and Health

These endpoints help verify that the system components are working correctly.

In [ ]:
# Basic system health check - this endpoint doesn't require authentication
system_health = make_request("GET", "/system-check", headers={})
system_health

In [ ]:
# Check database status
db_status = make_request("GET", "/debug/database", headers={})
db_status

In [ ]:
# Check vector database status
vector_status = make_request("GET", "/debug/vector-status", headers={})
vector_status

<a id="auth"></a>
## 3. Authentication

Verifying that the API key authentication is working correctly.

In [263]:
# Get current user information
current_user = make_request("GET", "/api/v1/me")
current_user

Request: GET http://localhost:8080/api/v1/me
Status: 200


{'username': 'TELLME_DIRECT',
 'email': 'direct@example.com',
 'team_id': '11111111-1111-1111-1111-111111111111',
 'is_admin': True,
 'id': '9058dc88-b824-4d26-a7f1-47b9609dec0a',
 'created_at': '2025-05-30T20:44:26.364786+00:00',
 'updated_at': None}

In [ ]:
# Test with invalid API key
invalid_auth = make_request("GET", "/api/v1/me", headers={"Authorization": "invalid_key"})
invalid_auth

<a id="teams"></a>
## 4. Team Management

Creating 5 model teams for our organization structure.

In [ ]:
# List existing teams using the debug endpoint
db_status = make_request("GET", "/debug/database")
print("Existing Teams:")
if isinstance(db_status, dict) and "teams" in db_status:
    teams_list = db_status.get("teams", [])
    for i, team in enumerate(teams_list):
        print(f"Team {i+1}: {team.get('name')} (ID: {team.get('id')})")
    print(f"Total: {len(teams_list)} teams")
else:
    teams_list = []
    print("No teams found or error retrieving teams")


In [ ]:
# Team creation process
# Define our 5 model teams focused on different aspects of Sereact AI Stuttgart
team_definitions = [
    {
        "name": "AI Research Team",
        "description": "Research team developing advanced deep learning models for robotics at Sereact AI Stuttgart"
    },
    {
        "name": "Robotics Development",
        "description": "Hardware and software integration team for robotic systems at Sereact AI Stuttgart"
    },
    {
        "name": "Computer Vision Team",
        "description": "Team focusing on visual perception and image processing for Sereact's automation solutions"
    },
    {
        "name": "Data Science Team",
        "description": "Analytics and data processing team supporting Sereact's AI applications"
    },
    {
        "name": "Integration Testing",
        "description": "Quality assurance team for Stuttgart's robotics deployment and validation"
    }
]

created_teams = []

# Create teams
for team_data in team_definitions:
    # Create a new team
    response = make_request("POST", "/api/v1/teams", json_data=team_data)
    print(f"Creating team: {team_data['name']}")
    
    # The actual creation might succeed even if we don't get the expected response format
    # So let's fetch the teams to check
    



### View Team Details

Let's view details for one of our newly created teams.

In [ ]:
# 5. User Management
# 
# This section creates 5 users for each of the first 5 teams in the database.
# The implementation accounts for the API returning a generic success message
# instead of the created user object.

# User data generator - 5 users for each team with guaranteed unique usernames
def generate_users_for_team(team_id, team_name):
    """Generate 5 users for a specific team with role-appropriate details and unique usernames"""
    
    # Define roles based on team name (for reference only)
    roles = {
        "AI Research Team": ["Lead AI Researcher", "Neural Network Specialist", "ML Engineer", "Research Assistant", "AI Ethics Advisor"],
        "Robotics Development": ["Robotics Lead", "Hardware Engineer", "Firmware Developer", "Mechanical Designer", "Testing Specialist"],
        "Computer Vision Team": ["Vision Lead", "Image Processing Engineer", "CV Algorithm Developer", "3D Perception Specialist", "Camera Systems Engineer"],
        "Data Science Team": ["Chief Data Scientist", "Data Engineer", "Analytics Specialist", "ML Operations", "Data Visualization Expert"],
        "Integration Testing": ["QA Lead", "Systems Tester", "Automation Engineer", "Performance Analyst", "Documentation Specialist"]
    }
    
    # Names for users (professional names)
    first_names = ["Alexander", "Emma", "Michael", "Sophia", "Daniel"]
    last_names = ["Schmidt", "Mueller", "Fischer", "Becker", "Schulz"]
    
    # Generate 5 users for the team
    users = []
    team_roles = roles.get(team_name, ["Specialist", "Engineer", "Analyst", "Developer", "Manager"])
    
    # Create a simple team name slug for username uniqueness
    team_slug = team_name.lower().replace(' ', '_')
    
    # Get current timestamp to ensure uniqueness
    import time
    timestamp = int(time.time())
    
    for i in range(5):
        first_name = first_names[i]
        last_name = last_names[i]
        
        # Generate a username based on first name, last name and team
        # Keep it short but unique by using just the team slug without timestamp
        # This makes it more readable in logs and database
        username = f"{first_name.lower()}.{last_name.lower()}_{team_slug}"
        email = f"{username}@sereact-ai.com"
        
        # Only include fields that exist in the User model
        user_data = {
            "username": username,
            "email": email,
            "team_id": str(team_id),  # Ensure team_id is a string
            "is_admin": False  # Regular users, not admins
        }
        
        users.append(user_data)
    
    return users

# Retrieve available teams from the database
print("Retrieving teams from database...")
db_status = make_request("GET", "/debug/database")
available_teams = []

if isinstance(db_status, dict) and "teams" in db_status:
    available_teams = db_status.get("teams", [])
    print(f"Found {len(available_teams)} teams in the database")
    
    # Display the first few teams
    for i, team in enumerate(available_teams[:5]):
        print(f"Team {i+1}: {team.get('name')} (ID: {team.get('id')})")
else:
    print("Error retrieving teams from database")

# Use only the first 5 teams to create users
teams_to_use = available_teams[:5]
print(f"\nWill create users for {len(teams_to_use)} teams")

# Create users for each team - Modified for API that returns generic success message
created_users_by_team = {}
all_attempted_creations = 0

for team in teams_to_use:
    team_id = team['id']
    team_name = team['name']
    
    user_data_list = generate_users_for_team(team_id, team_name)
    team_users_created = []
    
    print(f"\nCreating users for team: {team_name}")
    
    for user_data in user_data_list:
        username = user_data['username']
        print(f"  Creating user: {username}")
        
        response = make_request("POST", "/api/v1/users", json_data=user_data)
        all_attempted_creations += 1
        
        # Consider any response successful as our API always returns a generic message
        team_users_created.append(username)
        if isinstance(response, dict) and 'message' in response:
            print(f"  → Response: {response['message']}")
        else:
            print(f"  → Response: {response}")
    
    created_users_by_team[team_name] = team_users_created
    print(f"  Attempted to create {len(team_users_created)} users for {team_name}")

print(f"\nTotal user creation attempts: {all_attempted_creations}")

# Verify users in database
print("\nVerifying users in database...")
db_status = make_request("GET", "/debug/database")

if isinstance(db_status, dict) and "users_count" in db_status:
    print(f"Total users in database: {db_status['users_count']}")
    
    if db_status["users_count"] > 0 and "users" in db_status:
        print("\nSample users:")
        for i, user in enumerate(db_status["users"][:10]):  # Show first 10 users
            print(f"{i+1}. {user.get('username')} (Team ID: {user.get('team_id')})")
else:
    print("Error retrieving user data from database")

# Try to query users API endpoint directly
print("\nTrying to get users via API endpoint:")
users_response = make_request("GET", "/api/v1/users")

if isinstance(users_response, dict) and 'message' in users_response:
    print(f"API returned message: {users_response['message']}")
    print("This is expected due to the API serialization issue discussed previously.")
    print("Users are being created successfully but the API is returning a generic message.")
elif isinstance(users_response, list):
    print(f"API returned {len(users_response)} users")
    for i, user in enumerate(users_response[:5]):
        print(f"{i+1}. {user}")
else:
    print(f"API returned unexpected response: {users_response}")

print("\nUser creation process completed. Use the debug endpoint to verify users exist in the database.")


In [ ]:
# List all users
all_users = make_request("GET", "/api/v1/users")
if isinstance(all_users, list):
    print(f"Total users in system: {len(all_users)}")
    # Display first 5 users as example
    print(json.dumps(all_users, indent=2))

IndentationError: unindent does not match any outer indentation level (<string>, line 6)

In [ ]:
# the all users variable contains the list of all users in the system
db_status = make_request("GET", "/debug/database")
if isinstance(db_status, dict) and "users_count" in db_status:
    print(f"Total users in database: {db_status['users_count']}")
    
    if db_status["users_count"] > 0 and "users" in db_status:
        print("\nSample users:")
        for i, user in enumerate(db_status["users"][:10]):  # Show first 10 users
            print(f"{i+1}. {user.get('username')} (Team ID: {user.get('team_id')})")


<a id="apikeys"></a>
## 6. API Key Management

Creating API keys for each user.

In [ ]:
# Obter um usuário real do banco de dados
print("Obtendo usuários existentes do banco de dados...")
db_status = make_request("GET", "/debug/database")
existing_users = []

if isinstance(db_status, dict) and "users" in db_status:
    existing_users = db_status.get("users", [])
    print(f"Encontrados {len(existing_users)} usuários no banco de dados")
    
    # Mostrar alguns usuários para referência
    for i, user in enumerate(existing_users[:5]):
        print(f"Usuário {i+1}: {user.get('username')} (ID: {user.get('id')})")
else:
    print("Erro ao obter usuários do banco de dados")

# Selecionar um usuário para criar chave de API
if existing_users:
    user = existing_users[0]  # Usar o primeiro usuário
    user_id = user['id']
    username = user['username']
    
    print(f"\nCriando chave de API para: {username} (ID: {user_id})")
    
    # Dados para a chave de API
    api_key_data = {
        "name": f"API Key for {username}",
        "expires_at": None  # Sem expiração
    }
    
    # Tentar criar a chave de API
    response = make_request("POST", "/api/v1/api-keys", json_data=api_key_data)
    
    print(f"Resposta: {response}")
    
    # Verificar se houve sucesso ou falha
    if isinstance(response, dict) and 'id' in response:
        print(f"Chave de API criada com sucesso: {response}")
    else:
        print(f"Falha ao criar chave de API: {response}")
else:
    print("Não há usuários disponíveis para criar chaves de API")


In [ ]:
# List API keys
api_keys = make_request("GET", "/api/v1/api-keys")
api_keys